In [14]:
from tensorflow.keras.layers import Flatten, Dense, Input, Lambda, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img,ImageDataGenerator, img_to_array
import tensorflow as tf

import numpy as np
from glob import glob
import os
import matplotlib.pyplot as plt 

In [3]:
IMAGE_SIZE = [150,150]
train_path = ''
valid_path = ''

In [ ]:
# set your seeds to match each other, so your training and validation sets don't overlap
# BATCH_SIZE = 32
# IMG_SIZE = [150,150]
# directory = "dataset/"
# train_dataset = image_dataset_from_directory(directory,
#                                              shuffle=True,
#                                              batch_size=BATCH_SIZE,
#                                              image_size=IMG_SIZE,
#                                              validation_split=0.2,
#                                              subset='training',
#                                              seed=42)
# validation_dataset = image_dataset_from_directory(directory,
#                                              shuffle=True,
#                                              batch_size=BATCH_SIZE,
#                                              image_size=IMG_SIZE,
#                                              validation_split=0.2,
#                                              subset='validation',
#                                              seed=42)

In [ ]:
# class_names = train_dataset.class_names

# plt.figure(figsize=(10, 10))
# for images, labels in train_dataset.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_names[labels[i]])
#         plt.axis("off")

In [ ]:
# load the pretrained weights from ImageNet by specifying weights='imagenet'
resent_model = ResNet50V2(include_top=False, weights='imagenet', input_shape=IMAGE_SIZE + [3])
resent_model.summary()

In [8]:
# If we want, we can also only freeze first 3/5 portion of layers
# for layer in resent_model.layers:
#     layer.trainable = False
# nb_layers = len(resent_model.layers)
# print(resent_model.layers[nb_layers - 2].name)
# print(resent_model.layers[nb_layers - 1].name)

In [ ]:
## another way##
# # Freeze all the layers before the `fine_tune_at` layer
# for layer in base_model.layers[:fine_tune_at]:
#     #print('Layer ' + layer.name + ' frozen.')
#     layer.trainable = None
    
resent_model.trainable = True
for layer in resent_model.layers:
    if layer.name == 'conv3_block4_out':
        break
    layer.trainable = False
    #print('Layer ' + layer.name + ' frozen.')

In [9]:
# Add the new Binary classification layers
# use global avg pooling to summarize the info in each channel
x = GlobalAveragePooling2D()(resent_model.output)
#include dropout with probability of 0.2 to avoid overfitting
x = Dropout(0.5)(x)
# x = Flatten()(x)
# x = Dense(512,activation='relu')(x)
x = Dense(512,activation='relu', kernel_regularizer='l2')(x)
x = Dense(2,activation='softmax')(x)

model = Model(inputs=resent_model.input, outputs=x)

In [ ]:
model.summary()

In [ ]:
# Define a BinaryCrossentropy loss function. Use from_logits=True
loss_function= tf.keras.losses.BinaryCrossentropy(from_logits=True)
# Define an Adam optimizer with a learning rate of 0.1 * base_learning_rate
optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate*0.1)# 0.001
# Use accuracy as evaluation metric
metrics=['accuracy']

In [11]:
model.compile(
    optimizer=optimizer,
    loss= loss_function, #CHANGE
    metrics = metrics
)

In [12]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    horizontal_flip = True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
)

test_datagen = ImageDataGenerator(
    rescale = 1./255
)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (150,150),
                                                 batch_size = 128,
                                                 class_mode = 'categorical')

In [ ]:
testing_set = test_datagen.flow_from_directory(valid_path,
                                               target_size = (150,150),
                                               batch_size = 128,
                                               class_mode = 'categorical')

In [ ]:
# fine_tune_epochs = 5
# total_epochs =  initial_epochs + fine_tune_epochs

# history_fine = model.fit(train_dataset,
#                          epochs=total_epochs,
#                          initial_epoch=history.epoch[-1],
#                          validation_data=validation_dataset)

In [ ]:
hist = model.fit(training_set,
                validation_data = testing_set,
                epochs = 20,
                steps_per_epoch=len(training_set),
                validation_steps=len(testing_set))

In [ ]:
# plot the loss
plt.plot(hist.history['loss'], label='train loss')
plt.plot(hist.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(hist.history['accuracy'], label='train acc')
plt.plot(hist.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
from tensorflow.keras.models import load_model

model.save('ResNet_model.h5')